In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
tok_path = "/home/murilo/RelNetCare/data/raw/lora/tokenizer.pkl"
with open(tok_path, "rb") as f:
    tokenizer = pickle.load(f)

print(f"Tokenizer loaded from '{tok_path}'")


In [ ]:
import pandas as pd

turn_count = 2

if turn_count is None:
    data_path = '/home/murilo/RelNetCare/data/processed/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-train-dev.json'
else:
    data_path = f'/home/murilo/RelNetCare/data/processed/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxTrnCp{turn_count}/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxTrnCp{turn_count}-train-dev.json'

df = pd.read_json(data_path, orient='records')

print("JSON list loaded into DataFrame!")

df['prompt'] = df['conversations'].apply(lambda x: x[0].get('value') if x else None)
df['reply'] = df['conversations'].apply(lambda x: x[1].get('value') if x else None)
df['prompt_reply'] = df.apply(lambda row: "A system that extracts entities and their relations from dialogues. It returns results in jsonl format. USER: " + row['prompt'] + " ASSISTANT: " + row['reply'], axis=1)

from tqdm import tqdm
tqdm.pandas()

# Tokenize 'prompt_reply' and get the token count
df['prompt_reply_tokens'] = df['prompt_reply'].progress_apply(lambda x: tokenizer.tokenize(x))
df['token_count'] = df['prompt_reply_tokens'].progress_apply(len)

# Get the character count for comparison
df['char_count'] = df['prompt_reply'].progress_apply(len)
df



In [ ]:
import seaborn as sns
sns.set()
data_stem = data_path.split('/')[-1].replace('-train-dev', '').replace('-test', '').replace('.json', '')
ax = df.token_count.hist(bins=100, figsize=(12, 2), alpha=0.8)

print("Input + Output Token Count Distribution Token Count Distribution")
ax.set_title(f'Dataset: {data_stem}')
ax.axvline(x=512, color='r', linestyle='--', label='512 Limit')
ax.axvline(x=1024, color='g', linestyle='--', label='1024 Limit')

# Add legend to help interpret the lines
ax.legend()
ax.set_xlim([200, 1700])
ax.set_ylim([0, 300])


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

sns.set(style="white")

tqdm.pandas()
counts_list = [None, 2, 3, 5, 7, 10]
sample_counts = {}

for turn_count in counts_list:
    if turn_count is None:
        data_path = '/home/murilo/RelNetCare/data/processed/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-train-dev.json'
    else:
        data_path = f'/home/murilo/RelNetCare/data/processed/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxTrnCp{turn_count}/dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxTrnCp{turn_count}-train-dev.json'
    
    cnt_df = pd.read_json(data_path, orient='records')
    sample_counts[turn_count] = len(cnt_df)

# Sort dictionary by value
sorted_sample_counts = {k: v for k, v in reversed(sorted(sample_counts.items(), key=lambda item: item[1]))}

# Plotting
plt.figure(figsize=(5, 18), facecolor='white')
bars = plt.barh(list(map(str, sorted_sample_counts.keys())), list(sorted_sample_counts.values()), height=0.3)
plt.xlabel('Total Samples')
plt.ylabel('Dataset Turn Count')
plt.title('Total Samples per Dataset Turn Count')

# Adding data labels inside the bar
for bar in bars:
    width = bar.get_width()
    plt.text(width-0.15*max(sample_counts.values()), bar.get_y() + bar.get_height()/2, str(width), va='center', color='white')

plt.show()



In [172]:

# import re

# def extract_cls_number(s):
#     match = re.search(r'(\d+)cls', s)
#     if match:
#         return int(match.group(1))
#     return None

# import wandb

# # Initialize the API
# api = wandb.Api()

# # Your project name
# project_name = "huggingface"

# # Loop through runs in the project
# runs = list(api.runs(f"{project_name}"))
# runs = [r for r in runs if '4cls-rebalPairs4x' in r.config['data_stem']]
# display([r.config['data_stem'] for r in runs])
# # display({extract_cls_number(r.config['data_stem']) for r in runs})

# for run in runs:
#     run.config["exp_group"] = 'DialogREExtractTriplets_LrnCorrelRels'
#     run.update()


['dialog-re-llama-4cls-rebalPairs4x-rwrtKeys-instrC-mxTrnCp3-shfflDt']

['dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxCnvLgth2048',
 'dialog-re-llama-11cls-rebalPairs-rwrtKeys-instrC-mxCnvLgth2048']

11


In [ ]:
import re

def extract_intent_and_context_names(text):
    intent_pattern = r'intent {\s+name: "(.+?)"'
    context_pattern = r'output_contexts {\s+name: "(.+?)"'
    
    intent_names = re.findall(intent_pattern, text)
    context_names = re.findall(context_pattern, text)
    
    return {'intent_names': intent_names, 'context_names': context_names}

def extract_info(text):
    patterns = {
        'query_text': r'query_text: "(.+?)"',
        'fulfillment_texts': r'fulfillment_text: "(.+?)"',
        'display_names': r'display_name: "(.+?)"',
        'intent_detection_confidence': r'intent_detection_confidence: ([0-9.]+)',
        'intent_names': r'intent {\s+name: "(.+?)"',
        'context_names': r'output_contexts {\s+name: "(.+?)"',
    }
    
    extracted_info = {}
    for key, pattern in patterns.items():
        extracted_info[key] = re.findall(pattern, text)
        
    return extracted_info

info_dict = extract_info(tmp)

for k,v_list in info_dict.items():
    print(f"# {k}:")
    for v in v_list:
        if 'intent_names' in k or 'context' in k:
            v = '/'.join([x for x in v.split('/') if 'dialog' in x])
        print(v)
    print()

In [ ]:
tmp